In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

TESTING = False

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 50
if TESTING:
    epochs = 1

data_augmentation = not TESTING
num_predictions = 20

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
def create_model(filters, dropout_conv, extra_conv):
    model = Sequential()
    model.add(Conv2D(32, filters, padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, filters))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout_conv:
        model.add(Dropout(0.25))

    model.add(Conv2D(64, filters, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, filters))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout_conv:
        model.add(Dropout(0.25))
        
    if extra_conv:
        model.add(Conv2D(64, filters, padding='same'))
        model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model

In [4]:
def run_and_save(model_name, data_augmentation, score=False, 
                 filters=(3, 3), dropout_conv=True, extra_conv=False):
    # initiate RMSprop optimizer
    model = create_model(filters, dropout_conv, extra_conv)
    opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.0)

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4)
        
    # Save model and weights
    model_path = model_name
    model.save(model_path)
    print(f"Saved trained model at {model_path}")

    if score:
        # Score trained model.
        scores = model.evaluate(x_test, y_test, verbose=1)
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])

In [5]:
%%time
run_and_save("default_model.h5", data_augmentation)

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 134s 86ms/step - loss: 1.8479 - acc: 0.3180 - val_loss: 1.5200 - val_acc: 0.4541
Epoch 2/50
1563/1563 [==============================] - 131s 84ms/step - loss: 1.5683 - acc: 0.4276 - val_loss: 1.3744 - val_acc: 0.5013
Epoch 3/50
1563/1563 [==============================] - 128s 82ms/step - loss: 1.4516 - acc: 0.4741 - val_loss: 1.3161 - val_acc: 0.5266
Epoch 4/50
1563/1563 [==============================] - 132s 85ms/step - loss: 1.3779 - acc: 0.5077 - val_loss: 1.2458 - val_acc: 0.5523
Epoch 5/50
1563/1563 [==============================] - 132s 84ms/step - loss: 1.3133 - acc: 0.5291 - val_loss: 1.1547 - val_acc: 0.5940
Epoch 6/50
1563/1563 [==============================] - 131s 84ms/step - loss: 1.2558 - acc: 0.5534 - val_loss: 1.1030 - val_acc: 0.6108
Epoch 7/50
1563/1563 [==============================] - 125s 80ms/step - loss: 1.1987 - acc: 0.5724 - val_loss: 1.0625 - val_acc: 0.6231
Epoch 

In [6]:
%%time
run_and_save("filters_5_5.h5", data_augmentation,
            filters=(5,5))

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 229s 147ms/step - loss: 1.8548 - acc: 0.3128 - val_loss: 1.6396 - val_acc: 0.3917
Epoch 2/50
1563/1563 [==============================] - 229s 146ms/step - loss: 1.5608 - acc: 0.4289 - val_loss: 1.4303 - val_acc: 0.4842
Epoch 3/50
1563/1563 [==============================] - 229s 146ms/step - loss: 1.4195 - acc: 0.4872 - val_loss: 1.3314 - val_acc: 0.5288
Epoch 4/50
1563/1563 [==============================] - 229s 146ms/step - loss: 1.3199 - acc: 0.5299 - val_loss: 1.3516 - val_acc: 0.5286
Epoch 5/50
1563/1563 [==============================] - 228s 146ms/step - loss: 1.2371 - acc: 0.5565 - val_loss: 1.1748 - val_acc: 0.5789
Epoch 6/50
1563/1563 [==============================] - 228s 146ms/step - loss: 1.1689 - acc: 0.5823 - val_loss: 1.1504 - val_acc: 0.5963
Epoch 7/50
1563/1563 [==============================] - 227s 146ms/step - loss: 1.1195 - acc: 0.6054 - val_loss: 0.9484 - val_acc: 0.6612

In [7]:
%%time
run_and_save("extra_conv.h5", data_augmentation,
            filters=(5,5), extra_conv=True)

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 241s 154ms/step - loss: 1.8418 - acc: 0.3122 - val_loss: 1.5355 - val_acc: 0.4327
Epoch 2/50
1563/1563 [==============================] - 240s 154ms/step - loss: 1.5266 - acc: 0.4400 - val_loss: 1.4411 - val_acc: 0.4865
Epoch 3/50
1563/1563 [==============================] - 240s 154ms/step - loss: 1.3846 - acc: 0.5007 - val_loss: 1.2609 - val_acc: 0.5468
Epoch 4/50
1563/1563 [==============================] - 240s 154ms/step - loss: 1.2775 - acc: 0.5412 - val_loss: 1.1235 - val_acc: 0.6014
Epoch 5/50
1563/1563 [==============================] - 240s 153ms/step - loss: 1.2005 - acc: 0.5703 - val_loss: 1.0075 - val_acc: 0.6455
Epoch 6/50
1563/1563 [==============================] - 240s 153ms/step - loss: 1.1217 - acc: 0.6007 - val_loss: 1.1103 - val_acc: 0.6225
Epoch 7/50
1563/1563 [==============================] - 239s 153ms/step - loss: 1.0629 - acc: 0.6239 - val_loss: 1.0351 - val_acc: 0.6423

In [8]:
%%time
run_and_save("no_dropout_conv.h5", data_augmentation,
            filters=(5,5), extra_conv=True, dropout_conv=False)

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 236s 151ms/step - loss: 1.7777 - acc: 0.3402 - val_loss: 1.6154 - val_acc: 0.4312
Epoch 2/50
1563/1563 [==============================] - 236s 151ms/step - loss: 1.4827 - acc: 0.4612 - val_loss: 1.4274 - val_acc: 0.4916
Epoch 3/50
1563/1563 [==============================] - 237s 152ms/step - loss: 1.3316 - acc: 0.5224 - val_loss: 1.2847 - val_acc: 0.5415
Epoch 4/50
1563/1563 [==============================] - 237s 152ms/step - loss: 1.2176 - acc: 0.5678 - val_loss: 1.1971 - val_acc: 0.5784
Epoch 5/50
1563/1563 [==============================] - 237s 152ms/step - loss: 1.1226 - acc: 0.6025 - val_loss: 0.9799 - val_acc: 0.6588
Epoch 6/50
1563/1563 [==============================] - 236s 151ms/step - loss: 1.0443 - acc: 0.6303 - val_loss: 0.9455 - val_acc: 0.6712
Epoch 7/50
1563/1563 [==============================] - 236s 151ms/step - loss: 0.9786 - acc: 0.6549 - val_loss: 0.9643 - val_acc: 0.6627